In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d "abdullahkhanuet22/eggs-images-classification-damaged-or-not"

100% 100M/100M [00:01<00:00, 89.8MB/s] 
100% 100M/100M [00:01<00:00, 69.4MB/s]


In [ ]:
!unzip "eggs-images-classification-damaged-or-not.zip"

Archive:  eggs-images-classification-damaged-or-not.zip
  inflating: Eggs Classification/Damaged/damaged_1.jpg  
  inflating: Eggs Classification/Damaged/damaged_10.jpg  
  inflating: Eggs Classification/Damaged/damaged_100.jpg  
  inflating: Eggs Classification/Damaged/damaged_101.jpg  
  inflating: Eggs Classification/Damaged/damaged_102.jpg  
  inflating: Eggs Classification/Damaged/damaged_103.jpg  
  inflating: Eggs Classification/Damaged/damaged_104.jpg  
  inflating: Eggs Classification/Damaged/damaged_105.jpg  
  inflating: Eggs Classification/Damaged/damaged_106.jpg  
  inflating: Eggs Classification/Damaged/damaged_107.jpg  
  inflating: Eggs Classification/Damaged/damaged_108.jpg  
  inflating: Eggs Classification/Damaged/damaged_109.jpg  
  inflating: Eggs Classification/Damaged/damaged_11.jpg  
  inflating: Eggs Classification/Damaged/damaged_110.jpg  
  inflating: Eggs Classification/Damaged/damaged_111.jpg  
  inflating: Eggs Classification/Damaged/damaged_112.jpg  
  in

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
data = []
labels = []

dataset_dir = "Eggs Classification/"
for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    for filename in os.listdir(category_dir):
        img_path = os.path.join(category_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (24, 32))  # Resize image if necessary
        data.append(img)
        labels.append(category)

# Convert lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Split the dataset into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
# Reshape and normalize image data
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode labels as integers
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.fit_transform(test_labels)

# Convert integer labels to one-hot encoded arrays
train_labels_one_hot = keras.utils.to_categorical(train_labels_encoded, num_classes=2)
test_labels_one_hot = keras.utils.to_categorical(test_labels_encoded, num_classes=2)

In [ ]:
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 24, 3)),
    layers.MaxPooling2D(((2,2))),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Quantity to be monitored (validation loss)
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the best model weights based on the monitoring quantity
)

In [ ]:
model.fit(train_data,
          train_labels_one_hot,
          epochs=20,
          batch_size=32,
          validation_data=(test_data, test_labels_one_hot),
          callbacks=[early_stopping],
          verbose=1)

Epoch 1/20
20/20 [==============================] - 1s 57ms/step - loss: 0.2192 - accuracy: 0.9087 - val_loss: 0.2788 - val_accuracy: 0.8679
Epoch 2/20
20/20 [==============================] - 1s 53ms/step - loss: 0.1740 - accuracy: 0.9339 - val_loss: 0.2883 - val_accuracy: 0.8553
Epoch 3/20
20/20 [==============================] - 1s 57ms/step - loss: 0.1583 - accuracy: 0.9402 - val_loss: 0.2823 - val_accuracy: 0.8491
Epoch 4/20
20/20 [==============================] - 1s 63ms/step - loss: 0.1496 - accuracy: 0.9402 - val_loss: 0.2660 - val_accuracy: 0.8868
Epoch 5/20
20/20 [==============================] - 1s 54ms/step - loss: 0.1332 - accuracy: 0.9480 - val_loss: 0.3886 - val_accuracy: 0.8428
Epoch 6/20
20/20 [==============================] - 1s 47ms/step - loss: 0.1286 - accuracy: 0.9528 - val_loss: 0.3176 - val_accuracy: 0.8491
Epoch 7/20
20/20 [==============================] - 1s 36ms/step - loss: 0.1161 - accuracy: 0.9559 - val_loss: 0.2595 - val_accuracy: 0.8805
Epoch 8/20
20

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels_one_hot)
print('Test accuracy:', test_acc)

5/5 [==============================] - 0s 12ms/step - loss: 0.4321 - accuracy: 0.8050
Test accuracy: 0.805031418800354
